<a href="https://colab.research.google.com/github/yuriborg/dsmkt/blob/main/churn_data_leakage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display
import os, sys
import time
import datetime
import warnings
import pickle
import re
import unicodedata

plt.rcParams['axes.facecolor'] = 'white'
dt = datetime.datetime.now().strftime('%d%m%Y_%H%M%S')

#import missingno as msno
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

from sklearn.utils.class_weight import compute_sample_weight

from sklearn import preprocessing

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, recall_score, precision_score, f1_score, fbeta_score

from sklearn import tree

from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from sklearn import tree

from scipy.stats import chi2_contingency


from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import cohen_kappa_score, roc_auc_score
import statistics
from sklearn.linear_model import SGDClassifier


from scipy.stats import ks_2samp

from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SVMSMOTE
from imblearn.pipeline import Pipeline

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV


from sklearn.decomposition import PCA

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF, Matern, RationalQuadratic, ExpSineSquared

from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

from imbalanced_ensemble.metrics import classification_report_imbalanced


from sklearn.utils import class_weight
from sklearn.model_selection import cross_validate
from sklearn.pipeline import make_pipeline

from imblearn.ensemble import BalancedBaggingClassifier
from imblearn.over_sampling import RandomOverSampler
from sklearn.ensemble import BaggingClassifier
from sklearn.experimental import enable_hist_gradient_boosting  # noqa
from sklearn.ensemble import HistGradientBoostingClassifier
from imblearn.ensemble import BalancedBaggingClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.pipeline import make_pipeline as make_pipeline_with_sampler

import xgboost as xgb

from collections import Counter
import numpy as np
from imblearn import FunctionSampler


import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Salvar Plots no Bucket


In [ ]:
def find_bucket_obj(sub_str):
    s3 = boto3.resource(
        service_name='s3',
        region_name='us-east-1',
        aws_access_key_id= access_key,
        aws_secret_access_key= secret_access_key
    )
    for obj in s3.Bucket("sami-data-platform-s3-dev-sandbox-data-science").objects.all():
        if sub_str in obj._key:
            print(obj)


def save_html_plot_in_bucket(
    filename,
    my_bucket = "sami-data-platform-s3-dev-sandbox-data-science",
    path = '/dbfs/FileStore/shared_uploads/pedro.bloss@samisaude.com/plots_churn'
):
    s3 = boto3.resource(
        service_name='s3',
        region_name='us-east-1',
        aws_access_key_id=access_key,
        aws_secret_access_key=secret_access_key
    )
    input_file = os.path.join(
        path,
        filename
    )
    s3.Bucket(my_bucket).upload_file(input_file, filename)

# AWS Creds



In [ ]:
from cryptography.fernet import Fernet
import os
import shutil

def delete_folder_files(path):
    shutil.rmtree(path)
def delete_file(path, filename):
    os.remove(os.path.join(path, filename))

def encode_aws_keys(
    access_key,
    secret_access_key,
    path = '/dbfs/FileStore/shared_uploads/pedro.bloss@samisaude.com',
    temp_file = 'temp.csv',
    encoded_file = 'encoded_creds_pedro.csv'
):
    # setting paths and names
    temp_path = os.path.join(path, temp_file)
    encoded_path = os.path.join(path, encoded_file)

    # generating key
    key = Fernet.generate_key()
    fernet = Fernet(key)

    # creating dataframe and saving
    df_creds = pd.DataFrame({'access_key': access_key,
                            'secret_access_key': secret_access_key},
                            index=[0])
    df_creds.to_csv(temp_path)

    # reading data
    with open(temp_path, 'rb') as f:
        data = f.read()

    # encrypting data and saving
    encrypted_data = fernet.encrypt(data)
    with open(encoded_path, 'wb') as f:
        f.write(encrypted_data)

    delete_file(path, temp_file)

    return key

def get_aws_creds(
    key,
    path = '/dbfs/FileStore/shared_uploads/pedro.bloss@samisaude.com',
    encoded_file = 'encoded_creds_pedro.csv',
    decoded_file = 'decoded_creds_pedro.csv'
):
    # setting paths and names
    encoded_path = os.path.join(path, encoded_file)
    decoded_path = os.path.join(path, decoded_file)

    # opening the encrypted file
    with open(encoded_path, 'rb') as enc_file:
        encrypted = enc_file.read()

    # decrypting the file
    fernet = Fernet(key)
    decrypted = fernet.decrypt(encrypted)

    # writing the decoded creds
    with open(decoded_path, 'wb') as dec_file:
        dec_file.write(decrypted)
    df_dec = pd.read_csv(decoded_path)

    delete_file(path, decoded_file)

    return df_dec['access_key'].values[0], df_dec['secret_access_key'].values[0]

# access_key, secret_access_key = get_aws_creds(
#     key=,
#     path = '/dbfs/FileStore/shared_uploads/pedro.bloss@samisaude.com',
#     encoded_file = 'encoded_creds_pedro.csv'
# )

# Data Extraction

In [ ]:
df = load_csv_from_bucket('churn_preprocessed_18102022.csv')
labels = ['contract_id', 'e_desligado']
features = list(set(df.columns) - set(labels))
df = df[labels + features]
df

# Classifiers

In [ ]:
# import lightgbm as lgb

def get_classifiers(
    RANDOM_STATE = 42
):

    classifiers = {
        'model': [
                    # 'BalancedBagging+RandomOverSampler',
                    # 'Bagging+DecisionTree',
                    # 'BaggingClassifier',
                    # 'BalancedBagging+SMOTE',
                    # 'BalancedBagging+roughly_balanced_bagging',

                    'Balanced bag of histogram gradient boosting',

                    # 'HistGradientBoostingClassifier',
                    'Balanced RandomForest',
                    # 'BalancedBagging+RandomUnderSample',
                    'Undersampling RandomForest',
                    'Undersampling LogisticRegression',
                    # 'RandomForest+BalancedClassWeights',
                    'Undersampling XGBoost',
                    'Oversampling XGBoost',
                    # 'XGBoost',
                    'Bagging+KNN10',
                    # 'Bagging LinearSVC',
                    # 'LinearSVC'

                    'LightGBM',
                    'CatBoost'
        ],
        'classifier':[
                    # BalancedBaggingClassifier(
                    #                             sampler=RandomOverSampler(),
                    #                             n_estimators=100),
                    # BaggingClassifier(
                    #                     base_estimator=DecisionTreeClassifier(),
                    #                     random_state=RANDOM_STATE,
                    #                     n_estimators=100),
                    # BaggingClassifier(n_estimators=100),
                    # BalancedBaggingClassifier(sampler=SMOTE(),
                    #                             n_estimators=100),
                    # BalancedBaggingClassifier(
                    #     sampler=FunctionSampler(func=roughly_balanced_bagging, kw_args={"replace": True}),
                    #     n_estimators=25
                    # ),

                    make_pipeline(
                        BalancedBaggingClassifier(
                            base_estimator=HistGradientBoostingClassifier(random_state=RANDOM_STATE),
                            n_estimators=10,
                            random_state=RANDOM_STATE,
                            n_jobs=2,
                        ),
                    ),

                    # HistGradientBoostingClassifier(random_state=RANDOM_STATE,
                    #                                 learning_rate=0.9),
                    make_pipeline(
                      BalancedRandomForestClassifier(random_state=RANDOM_STATE,
                                                     n_jobs=2,
                                                    n_estimators=50),
                  ),
                    # BalancedBaggingClassifier(sampler=RandomUnderSampler(),
                    #                             n_estimators=25),
                    make_pipeline_with_sampler(
                        RandomUnderSampler(random_state=RANDOM_STATE),
                        RandomForestClassifier(random_state=RANDOM_STATE,
                                                n_jobs=2),
                    ),
                    make_pipeline_with_sampler(
                        RandomUnderSampler(random_state=RANDOM_STATE),
                        LogisticRegression(max_iter=1000),
                    ),
                    # RandomForestClassifier(random_state=RANDOM_STATE,
                    #                         class_weight='balanced',
                    #                         n_jobs=2,
                    #                         n_estimators=50),
                    make_pipeline_with_sampler(
                        RandomUnderSampler(random_state=RANDOM_STATE),
                        xgb.XGBClassifier(),
                    ),
                    make_pipeline_with_sampler(
                        RandomOverSampler(random_state=RANDOM_STATE),
                        xgb.XGBClassifier(),
                    ),
                    # xgb.XGBClassifier(scale_pos_weight=100),
                    BaggingClassifier(
                                        base_estimator=KNeighborsClassifier(n_neighbors=10),
                                        random_state=RANDOM_STATE
                                        ),
                    # BaggingClassifier(
                    #                     base_estimator=svm.SVC(kernel='linear',
                    #                                             C=1,
                    #                                             random_state=RANDOM_STATE,
                    #                                             probability = True),
                    #                     random_state=RANDOM_STATE
                    #                     ),
                    # svm.SVC(
                    #     kernel='linear',
                    #     random_state = RANDOM_STATE,
                    #     class_weight='balanced',
                    #     gamma='scale',
                    #     C=1.1,
                    #    probability=True
                    # ),
                    LGBMClassifier(random_state = RANDOM_STATE),
                    CatBoostClassifier(random_state = RANDOM_STATE)
        ],
        'randsearch_params': [
                        # # 'BalancedBagging+RandomOverSampler',
                        # {'n_estimators': [10, 25, 50, 100]},

                        # # 'Bagging+DecisionTree'
                        # {'n_estimators': [10, 25, 50, 100]},

                        # # BaggingClassifier
                        # {'n_estimators': [10, 25, 50, 100]},

                        # # BalancedBagging+SMOTE
                        # {'n_estimators': [10, 25, 50, 100]},

                        # # BalancedBagging+roughly_balanced_bagging
                        # {'n_estimators': [10, 25, 50, 100]},


                        # 'Balanced bag of histogram gradient boosting'
                        {'learning_rate': [0.0, 0.1, 0.5, 0.9, 1.0]},


                        # # HistGradientBoostingClassifier
                        # {'learning_rate': [0.0, 0.1, 0.5, 0.9, 1.0]},

                        # BalancedRandomForestClassifier
                        {'n_estimators': [10, 50, 100, 150, 200]},

                        # # 'BalancedBagging+RandomUnderSample'
                        # {'n_estimators': [10, 25, 50, 100]},

                        # 'Undersampling RandomForest'
                        {'n_estimators': [10, 50, 100, 150, 200]},

                        # 'Undersampling LogisticRegression'
                        {'penalty': ['l1', 'l2', 'elasticnet'],
                        'tol': [1e-4, 1e-3],
                        'C': [1.0, 0.9, 1.1]},

                        # # 'RandomForest+BalancedClassWeights'
                        # {'n_estimators': [10, 50, 100, 150, 200]},

                        # 'Undersampling XGBoost'
                        {'scale_pos_weight': [10, 50, 90, 99, 100]},

                        # 'Oversampling XGBoost'
                        {'scale_pos_weight': [10, 50, 90, 99, 100]},

                        # # XGBoost
                        # {'scale_pos_weight': [10, 50, 90, 99, 100]},

                        # 'Bagging+KNN10'
                        {'weights': ['uniform', 'distance'],
                        'n_neighbors': [3, 5, 8, 10, 15]},

                        # # # Bagging+LinearSVC
                        # # {'gamma': ['scale', 'auto'],
                        # # 'C': [1.0, 1.1, 1.5, 0.9]},

                        # # LinearSVC
                        # {'gamma': ['scale', 'auto'],
                        # 'C': [1.0, 1.1, 1.5, 0.9]},

                        # LightGBM params
                        {
                            'max_depth':[-1, 50, 100, 200],
                            'learning_rate': [0.001, 0.01, 0.1, 0.5, 0.9, 1.0, 1.2, 1.5],
                            'n_estimators': [100, 150, 200, 250, 500],
#                             'class_weight': [None, {0: 0.9, 1: 0.1}],
                            'scale_pos_weight':[None, 1.0, 0.1, 0.05]
                        },

                        # CatBoost
                        {
                            'max_depth':[-1, 50, 100, 200],
                            'learning_rate':[0.001, 0.01, 0.1, 0.5, 0.9, 1.0, 1.2, 1.5],
                            'n_estimators': [100, 150, 200, 250, 500],
                            'scale_pos_weight':[None, 1.0, 0.1, 0.05]
                        },
        ],
        'gridsearch_params': [
                        # # 'BalancedBagging+RandomOverSampler',
                        # {'n_estimators': [10, 25, 50, 75, 100,
                        #                     10, 25, 50, 75, 100],
                        # 'warm_start': [False]*5 + [True]*5},

                        # # 'Bagging+DecisionTree'
                        # {'n_estimators': [10, 25, 50, 75, 100,
                        #                     10, 25, 50, 75, 100],
                        # 'warm_start': [False]*5 + [True]*5},

                        # # 'BaggingClassifier'
                        # {'n_estimators': [10, 25, 50, 75, 100,
                        #                     10, 25, 50, 75, 100],
                        # 'warm_start': [False]*5 + [True]*5},

                        # # 'BalancedBagging+SMOTE'
                        # {'n_estimators': [10, 25, 50, 75, 100,
                        #                     10, 25, 50, 75, 100],
                        # 'warm_start': [False]*5 + [True]*5},

                        # # 'BalancedBagging+roughly_balanced_bagging'
                        # {'n_estimators': [10, 25, 50, 75, 100,
                        #                     10, 25, 50, 75, 100],
                        # 'warm_start': [False]*5 + [True]*5},


                        # 'Balanced bag of histogram gradient boosting'
                        {'loss': ['auto']*5,
                            'learning_rate': [0.0, 0.1, 0.5, 0.9, 1.0]},


                        # # HistGradientBoostingClassifier
                        # {'loss': ['auto']*5,
                        #     'learning_rate': [0.0, 0.1, 0.5, 0.9, 1.0]},

                        # BalancedRandomForestClassifier
                        {'n_estimators': [10, 50, 100, 150, 200]},

                        # # 'BalancedBagging+RandomUnderSample'
                        # {'n_estimators': [10, 25, 50, 75, 100,
                        #                     10, 25, 50, 75, 100],
                        # 'warm_start': [False]*5 + [True]*5},

                        # 'Undersampling RandomForest'
                        {'n_estimators': [10, 50, 100, 150, 200]},

                        # 'Undersampling LogisticRegression'
                        {'penalty': ['l1', 'l2', 'elasticnet']*3,
                        'C': [1.0, 1.0, 1.0,
                                0.9, 0.9, 0.9,
                                1.1, 1.1, 1.1]},

                        #     # 'RandomForest+BalancedClassWeights'
                        #     {'n_estimators': [10, 50, 100, 150, 200]},

                            # 'Undersampling XGBoost'
                            {'scale_pos_weight': [10, 50, 90, 99, 100]},

                            # 'Oversampling XGBoost'
                            {'scale_pos_weight': [10, 50, 90, 99, 100]},

                            # # XGBoost
                            # {'scale_pos_weight': [10, 50, 90, 99, 100]},

                            # 'Bagging+KNN10'
                            {'weights': ['uniform']*5 + ['distance']*5,
                            'n_neighbors': [3, 5, 8, 10, 15]*2},

                            # # # Bagging+LinearSVC
                            # # {'gamma': ['scale']*4 + ['auto']*4,
                            # # 'C': [1.0, 1.1, 1.5, 0.9]},

                            # # LinearSVC
                            # {'gamma': ['scale']*4 + ['auto']*4,
                            # 'C': [1.0, 1.1, 1.5, 0.9]},

                            # LGBM
                            {},

                            # CatBoost
                            {}
        ]
    }


    # for k in list(classifiers.keys()):
    #   print(f'key: {k}, len: {len(classifiers[k])}')

    return pd.DataFrame(classifiers)

df_classifiers = get_classifiers()
df_classifiers

# Função Teste de Modelos

In [ ]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score

import re
import unicodedata

def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii.decode('utf-8')


def filter_alphanum_str(x):
    return remove_accents(re.sub(r'\W+', '', x))

def get_features_labels_subset(df,
                               features,
                               p = 0.8,
                               target_label = 'e_desligado',
                               adjust_columns = False
                              ):
    t = df.copy()

    if adjust_columns:
        t.columns = list(map(
            lambda x: x.replace('.0', '').replace('>', 'maior').replace('<','menor').replace('[','').replace(']','').replace('-','').replace(',','_').replace('+','mais'),
                                   t.columns))
        t.columns = list(map(filter_alphanum_str, t.columns))

        features = list(map(
            lambda x: x.replace('.0', '').replace('>', 'maior').replace('<','menor').replace('[','').replace(']','').replace('-','').replace(',','_').replace('+','mais'),
                                   features))
        features = list(map(filter_alphanum_str, features))


    N = len(t)
    N_train = round(N * p)
    N_test = N - N_train

    X_train, X_test = t[features].loc[:N_train], t[features].loc[N_train:]
    y_train, y_test = t[target_label].loc[:N_train].values, t[target_label].loc[N_train:].values

    return X_train, y_train, X_test, y_test

def test_model_performance(
    X_train, y_train,
    X_test, y_test,
    clf,
    model_name,
    adjust_X_train_columns = False
):
    if adjust_X_train_columns:
        X_train.columns = list(map(
            lambda x: x.replace('.0', '').replace('>', 'maior').replace('<','menor').replace('[','').replace(']','').replace('-',''),
                                   X_train.columns))
        X_test.columns = list(map(
            lambda x: x.replace('.0', '').replace('>', 'maior').replace('<','menor').replace('[','').replace(']','').replace('-',''),
                                   X_test.columns))

        X_train.columns = list(map(filter_alphanum_str, X_train.columns))
        X_test.columns = list(map(filter_alphanum_str, X_test.columns))

    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)
    y_pred_proba = clf.predict_proba(X_test)

    report = pd.DataFrame(classification_report_imbalanced(y_test, y_pred, output_dict=True))

    d = {
        'metric': list(report[1].index[:5]),
        'values': list(map(lambda x: round(x, 3), report[1].values[:5])),
        'model': len(report[1].values[:5])*[model_name]
    }
    d['metric'].append('accuracy')
    d['values'].append(round(accuracy_score(y_true = y_test,
                                     y_pred = y_pred),
                             3)
                      )
    d['model'].append(model_name)
    report_churn = pd.DataFrame(d)

    return report_churn

In [ ]:
import math

def get_features_labels_balanced(
    df,
    features,
    p = 0.8,
    target_label = 'e_desligado',
   adjust_columns = False
):
    t = df.copy()

    # shuffling indexes
    t = t.sample(frac=1).reset_index(drop=True)

    if adjust_columns:
        t.columns = list(map(
            lambda x: x.replace('.0', '').replace('>', 'maior').replace('<','menor').replace('[','').replace(']','').replace('-','').replace(',','_').replace('+','mais'),
                                   t.columns))
        t.columns = list(map(filter_alphanum_str, t.columns))

        features = list(map(
            lambda x: x.replace('.0', '').replace('>', 'maior').replace('<','menor').replace('[','').replace(']','').replace('-','').replace(',','_').replace('+','mais'),
                                   features))
        features = list(map(filter_alphanum_str, features))

    # train data qtd
    N = len(t)
    N_train = math.ceil(N * p)
    N_test = N - N_train

    # initial balancement
    qtd_pos = len(t[t[target_label] == 1])
    qtd_neg = len(t[t[target_label] == 0])
    print('qtd_pos: {} ({:.3f}%), qtd_neg: {} ({:.3f}%)'.format(
        qtd_pos, (qtd_pos*100/(qtd_pos+qtd_neg)),
        qtd_neg, (qtd_neg*100/(qtd_pos+qtd_neg))
    ))
    # label proportion
    N_train_pos = math.ceil(qtd_pos * p)
    N_train_neg =  math.ceil(qtd_neg * p)
    N_tot = N_train_pos + N_train_neg
    print('N_train_pos: {} ({:.3f}%), N_train_neg: {} ({:.3f}%)'.format(
        N_train_pos, (N_train_pos*100/N_tot),
        N_train_neg, (N_train_neg*100/N_tot)
    ))
    # positive samples
    pos_train_data_sample = t[t[target_label] == 1].reset_index(drop=True).loc[:N_train_pos]

    X_train_pos = pos_train_data_sample[features]
    y_train_pos = pos_train_data_sample[target_label].values

    # negative samples
    neg_train_data_sample = t[t[target_label] == 0].reset_index(drop=True).loc[:N_train_neg]

    X_train_neg = neg_train_data_sample[features]
    y_train_neg = neg_train_data_sample[target_label].values


    # Training data
    X_train = pd.concat([X_train_pos, X_train_neg],
                       axis = 0)
    y_train = list(y_train_pos) + list(y_train_neg)


    # test data
    test_data_sample = t[
        (~t.index.isin(pos_train_data_sample.index)) &
        (~t.index.isin(neg_train_data_sample.index))
    ]
    X_test = test_data_sample[features]
    y_test = test_data_sample[target_label].values

    return X_train, np.array(y_train), X_test, np.array(y_test)


# Balanceamento dos Dados



In [ ]:
X_train, y_train, X_test, y_test = get_features_labels_balanced(df,
                                           features,
                                           p = 0.8,
                                          adjust_columns = True
                                   )

X_train

# Teste - Todas as Features

In [ ]:
features = list(set(df.columns) - set(labels))

feats_remove = [
    'churn_não churnou',
    'churn_churnou',
    'tempo_churn_meses',
    'prestador_mais_freq_nan',
#     'tipo_contato_nps_Membro - Cancelado',
#     'tipo_contato_nps_Contratante - Não membro'
]
features = list(set(features) - set(feats_remove))
features = list(filter(lambda x: 'não preenchido' not in x and x[-4:] != '_na',
                      features
                      ))
X_train, y_train, X_test, y_test = get_features_labels_balanced(df,
                                           features,
                                           p = 0.8,
                                          adjust_columns = True
                                   )

report_dummy_clf = test_model_performance(
    X_train, y_train,
    X_test, y_test,
    clf = DummyClassifier(),
    model_name = 'Dummy Classifier'
)
report_log_reg = test_model_performance(
    X_train, y_train,
    X_test, y_test,
    clf = LogisticRegression(),
    model_name = 'Logistic Reg'
)
report_rf = test_model_performance(
    X_train, y_train,
    X_test, y_test,
    clf = RandomForestClassifier(random_state = 42),
    model_name = 'Random Forest'
)
report_us_rf = test_model_performance(
    X_train, y_train,
    X_test, y_test,
    clf = df_classifiers[df_classifiers['model'] == 'Undersampling RandomForest']['classifier'].values[0],
    model_name = 'UnderSamp RandomForest'
)
report_brf = test_model_performance(
    X_train, y_train,
    X_test, y_test,
    clf = df_classifiers[df_classifiers['model'] == 'Balanced RandomForest']['classifier'].values[0],
    model_name = 'Balanced RandomForest'
)
report_us_xb = test_model_performance(
    X_train, y_train,
    X_test, y_test,
    clf = df_classifiers[df_classifiers['model'] == 'Undersampling XGBoost']['classifier'].values[0],
    model_name = 'UnderSamp XGBoost'
)
report_lgbm = test_model_performance(
    X_train, y_train,
    X_test, y_test,
    clf = df_classifiers[df_classifiers['model'] == 'LightGBM']['classifier'].values[0],
    model_name = 'LightGBM'
)
report_catboost = test_model_performance(
    X_train, y_train,
    X_test, y_test,
    clf = df_classifiers[df_classifiers['model'] == 'CatBoost']['classifier'].values[0],
    model_name = 'CatBoost'
)

reports = pd.concat([
                        report_dummy_clf,
                        report_log_reg,
                        report_rf,
                        report_us_rf,
                        report_brf,
                        report_us_xb,
                        report_lgbm,
                        report_catboost
                    ], axis = 0)

fig = px.line_polar(reports,
                        r="values",
                        theta='metric',
                       color = 'model',
                        line_close=True,
                        height = 400,
                        width = 700
                       )

fig.update_layout(
    title = 'Performances - previsão de churn ({} features)'.format(len(features)),
    title_x = 0.5,
#         title_y = 0.6,
    xaxis_title="variáveis",
    yaxis_title="minmax score",
    font=dict(
        family="Lato",
        size=12,
    ),
    legend = dict(
        title = 'Modelos iniciais'
    )
)
fig.show()

fig = px.bar(reports,
                     y="values",
                    x='metric',
                    color = 'model',
                    barmode = 'group',
                    text='values',
                    height = 400,
                    width = 1100
                       )

fig.update_layout(
    title = 'Performances - previsão de churn ({} features)'.format(len(features)),
    title_x = 0.5,
    yaxis_title="Performance",
    xaxis_title="Métrica",
    font=dict(
        family="Lato",
        size=16,
    ),
)
# out_path = '/dbfs/FileStore/shared_uploads/pedro.bloss@samisaude.com/plots_churn'
# titulo = 'remocao_dataleakage_19102022'
# out_file = os.path.join(out_path,
#                        'clf_results_{}.html'.format(
#                             titulo
#                             )
#                        )
# fig.write_html(out_file)
fig.show()

# Sem suspeitas de Data Leakage

In [ ]:
list(filter(lambda x: 'membro' in x.lower() and 'cancelado' in x.lower(), features))

In [ ]:
features2 = list(set(df.columns) - set(labels))

feats_remove = [
    'churn_não churnou',
    'churn_churnou',
    'tempo_churn_meses',
    'prestador_mais_freq_nan',
    'tipo_contato_nps_Membro - Cancelado',
    'tipo_contato_nps_Contratante - Não membro'
]
features2 = list(set(features2) - set(feats_remove))
features2 = list(filter(lambda x: 'não preenchido' not in x and x[-4:] != '_na',
                      features2
                      ))
# data_leakage_suspects = list(filter(lambda x: 'membro' in x.lower() and 'cancelado' in x.lower(),
#                                    df.columns))
data_leakage_suspects = list(filter(lambda x: 'cancel' in x.lower(), df.columns))

features2 = list(set(features2) - set(data_leakage_suspects))

list(filter(lambda x: 'membro' in x.lower() and 'cancelado' in x.lower(), features2))

In [ ]:
data_leakage_suspects

In [ ]:
X_train, y_train, X_test, y_test = get_features_labels_balanced(df,
                                           features2,
                                           p = 0.8,
                                          adjust_columns = True
                                   )

report_dummy_clf = test_model_performance(
    X_train, y_train,
    X_test, y_test,
    clf = DummyClassifier(),
    model_name = 'Dummy Classifier'
)
report_log_reg = test_model_performance(
    X_train, y_train,
    X_test, y_test,
    clf = LogisticRegression(),
    model_name = 'Logistic Reg'
)
report_rf = test_model_performance(
    X_train, y_train,
    X_test, y_test,
    clf = RandomForestClassifier(random_state = 42),
    model_name = 'Random Forest'
)
report_us_rf = test_model_performance(
    X_train, y_train,
    X_test, y_test,
    clf = df_classifiers[df_classifiers['model'] == 'Undersampling RandomForest']['classifier'].values[0],
    model_name = 'UnderSamp RandomForest'
)
report_brf = test_model_performance(
    X_train, y_train,
    X_test, y_test,
    clf = df_classifiers[df_classifiers['model'] == 'Balanced RandomForest']['classifier'].values[0],
    model_name = 'Balanced RandomForest'
)
report_us_xb = test_model_performance(
    X_train, y_train,
    X_test, y_test,
    clf = df_classifiers[df_classifiers['model'] == 'Undersampling XGBoost']['classifier'].values[0],
    model_name = 'UnderSamp XGBoost'
)
report_lgbm = test_model_performance(
    X_train, y_train,
    X_test, y_test,
    clf = df_classifiers[df_classifiers['model'] == 'LightGBM']['classifier'].values[0],
    model_name = 'LightGBM'
)
report_catboost = test_model_performance(
    X_train, y_train,
    X_test, y_test,
    clf = df_classifiers[df_classifiers['model'] == 'CatBoost']['classifier'].values[0],
    model_name = 'CatBoost'
)

reports2 = pd.concat([
                        report_dummy_clf,
                        report_log_reg,
                        report_rf,
                        report_us_rf,
                        report_brf,
                        report_us_xb,
                        report_lgbm,
                        report_catboost
                    ], axis = 0)

fig = px.line_polar(reports2,
                        r="values",
                        theta='metric',
                       color = 'model',
                        line_close=True,
                        height = 400,
                        width = 700
                       )

fig.update_layout(
    title = 'Performances - previsão de churn ({} features)'.format(len(features2)),
    title_x = 0.5,
#         title_y = 0.6,
    xaxis_title="variáveis",
    yaxis_title="minmax score",
    font=dict(
        family="Lato",
        size=12,
    ),
    legend = dict(
        title = 'Modelos iniciais'
    )
)
fig.show()

fig = px.bar(reports,
                     y="values",
                    x='metric',
                    color = 'model',
                    barmode = 'group',
                    text='values',
                    height = 400,
                    width = 1100
                       )

fig.update_layout(
    title = 'Performances - previsão de churn ({} features)'.format(len(features2)),
    title_x = 0.5,
    yaxis_title="Performance",
    xaxis_title="Métrica",
    font=dict(
        family="Lato",
        size=16,
    ),
)
# out_path = '/dbfs/FileStore/shared_uploads/pedro.bloss@samisaude.com/plots_churn'
# titulo = 'remocao_dataleakage_19102022'
# out_file = os.path.join(out_path,
#                        'clf_results_{}.html'.format(
#                             titulo
#                             )
#                        )
# fig.write_html(out_file)
fig.show()

# Comparação

In [ ]:
def get_mean_performance(reports,
                        teste):
    d = {
    'metric':[],
    'value': [],
    '%':[],
    'test':[]

    }
    metrics = list(reports['metric'].unique())
    for metric in metrics:
        d['metric'].append(metric)
        d['value'].append(reports[reports['metric']==metric].mean()[0])
        d['%'].append('{:.2f}%'.format(reports[reports['metric']==metric].mean()[0]*100))
        d['test'].append(teste)

    return pd.DataFrame(d)
get_mean_performance(reports, 'original')

In [ ]:
t = pd.concat([
            get_mean_performance(reports, 'original'),
            get_mean_performance(reports2, 'removendo variáveis suspeitas')
        ],
        axis=0).reset_index(drop=True)

fig = px.bar(t,
                     y="value",
                    x='metric',
                    color = 'test',
                    barmode = 'group',
                    text='%',
                    height = 400,
                    width = 1200
                       )

fig.update_layout(
    title = 'Analisando possibilidade de data leakage',
    title_x = 0.5,
    yaxis_title="Performance média",
    xaxis_title="Métrica",
    font=dict(
        family="Lato",
        size=16,
    ),
    legend = dict(title = 'Teste')
)
fig.show()

In [ ]:
models = list(reports['model'].unique())
metrics = list(reports['metric'].unique())

d_all = {
    'model':[],
    'metric_dif':[],
    'value_dif':[],
    'pct_dif':[]
}

for model in models:
    t1 = reports[reports['model']==model].copy()
    t1['teste'] = len(t1)*['original']

    t2 = reports2[reports2['model']==model].copy()
    t2['teste'] = len(t2)*['removendo variáveis suspeitas']

    t = pd.concat([t1, t2], axis=0).reset_index(drop=True)
    t['pct'] = list(map(lambda x: round(x*100, 3), t['values'].values))

    for metric in metrics:
        d_all['metric_dif'].append(metric)
        d_all['value_dif'].append(
            t1[t1['metric']==metric]['values'].values[0] - t2[t2['metric']==metric]['values'].values[0]
        )
        d_all['pct_dif'].append(
            '{:.1f}%'.format((t1[t1['metric']==metric]['values'].values[0] - t2[t2['metric']==metric]['values'].values[0])*100)
        )
        d_all['model'].append(model)

t_all = pd.DataFrame(d_all)

fig = px.bar(t_all,
                     y="value_dif",
                    x='metric_dif',
                    color = 'model',
                    barmode = 'group',
                    text='pct_dif',
                    height = 500,
                    width = 1400
                       )

fig.update_layout(
    title = 'Analisando possibilidade de data leakage (Diferenças em performance)',
    title_x = 0.5,
    yaxis_title="Diferença Performance",
    xaxis_title="Métrica (antes - depois)",
    font=dict(
        family="Lato",
        size=12,
    ),
    legend = dict(title = 'Teste')
)
fig.update_traces(textfont_size=18, textangle=0, textposition="outside", cliponaxis=False)

fig.show()

In [ ]:
models = list(reports['model'].unique())

for model in models:
    t1 = reports[reports['model']==model].copy()
    t1['teste'] = len(t1)*['original']

    t2 = reports2[reports2['model']==model].copy()
    t2['teste'] = len(t2)*['removendo variáveis suspeitas']

    t = pd.concat([t1, t2], axis=0).reset_index(drop=True)
    t['pct'] = list(map(lambda x: round(x*100, 3), t['values'].values))

    fig = px.bar(t,
                         y="values",
                        x='metric',
                        color = 'teste',
                        barmode = 'group',
                        text='pct',
                        height = 300,
                        width = 900
                           )

    fig.update_layout(
        title = 'Analisando possibilidade de data leakage ({})'.format(model),
        title_x = 0.5,
        yaxis_title="Performance",
        xaxis_title="Métrica",
        font=dict(
            family="Lato",
            size=16,
        ),
        legend = dict(title = 'Teste')
    )
    fig.show()